<a href="https://colab.research.google.com/github/schatz06/EPL448_Project/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Map Driver##

In [40]:
#from google.colab import drive
#drive.mount('/content/drive')

##Import Packages ## 

In [41]:
import numpy as np 
import pandas as pd 

##Read Data##

In [42]:
train_data = pd.read_excel('Data_Train.xlsx') # read training data
test_data  = pd.read_excel('Data_Test.xlsx') # read test data

Check for corrupted or empty records

In [43]:
print("Problematic records in training data: ",train_data.isnull().sum().sum())
print("Problematic records in test data: ",test_data.isnull().sum().sum())

Problematic records in training data:  0
Problematic records in test data:  0


##Train Data Preprocessing##

Reviews Normalization

In [44]:
train_data['Reviews'] = train_data.Reviews.apply(lambda r: float(r.split()[0])) # get only the review rule 
train_data['Reviews'] = (train_data.Reviews - train_data.Reviews.mean())/(train_data.Reviews.std()) # normalize reviews 

Ratings Normalization

In [45]:
train_data['Ratings'] = train_data.Ratings.apply(lambda r: float(r.split()[0].replace(',',''))) # get only the rating 
train_data['Ratings'] = (train_data.Ratings - train_data.Ratings.mean())/(train_data.Ratings.std()) # normalize rating 

Drop Title & Author & Synopsis & Genre columns

In [46]:
train_data.pop('Author')
train_data.pop('Title')
train_data.pop('Synopsis')
train_data.pop('Genre')

0         Action & Adventure (Books)
1         Cinema & Broadcast (Books)
2            International Relations
3       Contemporary Fiction (Books)
4              Photography Textbooks
                    ...             
6232            Anthropology (Books)
6233    Contemporary Fiction (Books)
6234                 Romance (Books)
6235      Action & Adventure (Books)
6236      Action & Adventure (Books)
Name: Genre, Length: 6237, dtype: object

Edition Normalization / Drop edition

In [47]:
#train_data['Edition'] = train_data.Edition.apply(lambda r: r.split(',')[0]) # get only the type of book e.g Paperback or Hardcover
#train_data= pd.get_dummies(train_data, columns= ['Edition'])
train_data.pop('Edition')

0       Paperback,– 10 Mar 2016
1        Paperback,– 7 Nov 2012
2       Paperback,– 25 Feb 1982
3        Paperback,– 5 Oct 2017
4       Hardcover,– 10 Oct 2006
                 ...           
6232     Paperback,– 8 Aug 2018
6233    Paperback,– 21 Nov 2016
6234     Paperback,– 8 Jun 2006
6235    Paperback,– 15 Jan 2015
6236    Paperback,– 21 Dec 2016
Name: Edition, Length: 6237, dtype: object

Book Category Normalization / Extract Categories

In [48]:
train_data= pd.get_dummies(train_data, columns= ['BookCategory'])

Finish training inputs(z)

In [49]:
z = train_data

In [50]:
train_data.head()

,Reviews,Ratings,Price,BookCategory_Action & Adventure,"BookCategory_Arts, Film & Photography","BookCategory_Biographies, Diaries & True Accounts",BookCategory_Comics & Mangas,"BookCategory_Computing, Internet & Digital Media","BookCategory_Crime, Thriller & Mystery",BookCategory_Humour,"BookCategory_Language, Linguistics & Writing",BookCategory_Politics,BookCategory_Romance,BookCategory_Sports
0,-0.442568,-0.186568,220.00,1,0,0,0,0,0,0,0,0,0,0
1,-0.593511,-0.146567,202.93,0,0,1,0,0,0,0,0,0,0,0
2,0.764977,-0.199902,299.00,0,0,0,0,0,0,1,0,0,0,0
3,-0.291625,-0.153234,180.00,0,0,0,0,0,1,0,0,0,0,0
4,1.066863,-0.233236,965.62,0,1,0,0,0,0,0,0,0,0,0


##Test Data Preprocessing##

Reviews Normalization

In [51]:
test_data['Reviews'] = test_data.Reviews.apply(lambda r: float(r.split()[0])) # get only the review rule 
test_data['Reviews'] = (test_data.Reviews - test_data.Reviews.mean())/(test_data.Reviews.std()) # normalize reviews 

Ratings Normalization

In [52]:
test_data['Ratings'] = test_data.Ratings.apply(lambda r: float(r.split()[0].replace(',',''))) # get only the rating 
test_data['Ratings'] = (test_data.Ratings - test_data.Ratings.mean())/(test_data.Ratings.std()) # normalize rating 

Drop Title & Author & Synopsis & Genre columns

In [53]:
test_data.pop('Author')
test_data.pop('Title')
test_data.pop('Synopsis')
test_data.pop('Genre')

0                            Short Stories (Books)
1                      Operating Systems Textbooks
2                                  Romance (Books)
3       Programming & Software Development (Books)
4                              Linguistics (Books)
                           ...                    
1555                                        Design
1556                  Children's Reference (Books)
1557                                Humour (Books)
1558                         United States History
1559                                  Dictionaries
Name: Genre, Length: 1560, dtype: object

Edition Normalization / Drop edition 

In [54]:
#test_data['Edition'] = test_data.Edition.apply(lambda r: r.split(',')[0]) # get only the type of book e.g Paperback or Hardcover
#test_data= pd.get_dummies(test_data, columns= ['Edition'])
test_data.pop('Edition')

0       Mass Market Paperback,– 1 Oct 1986
1          Paperback,– Import, 26 Apr 2018
2                  Paperback,– 17 Mar 2011
3                         Paperback,– 2016
4                  Paperback,– 26 Dec 2011
                       ...                
1555               Paperback,– 14 Apr 2011
1556                Paperback,– 8 May 2013
1557                Paperback,– 6 Sep 2011
1558               Paperback,– 22 Sep 2009
1559               Paperback,– 16 Sep 2016
Name: Edition, Length: 1560, dtype: object

Book Category Normalization / Extract Categories

In [55]:
test_data= pd.get_dummies(test_data, columns= ['BookCategory'])

Finish Test set Inputs

In [56]:
z_test = test_data
z_test.head()

,Reviews,Ratings,BookCategory_Action & Adventure,"BookCategory_Arts, Film & Photography","BookCategory_Biographies, Diaries & True Accounts",BookCategory_Comics & Mangas,"BookCategory_Computing, Internet & Digital Media","BookCategory_Crime, Thriller & Mystery",BookCategory_Humour,"BookCategory_Language, Linguistics & Writing",BookCategory_Politics,BookCategory_Romance,BookCategory_Sports
0,0.140178,5.627725,0,0,0,0,0,1,0,0,0,0,0
1,1.038851,-0.198467,0,0,0,0,1,0,0,0,0,0,0
2,1.038851,-0.180241,0,0,0,0,0,0,0,0,0,1,0
3,-0.309159,-0.137714,0,0,0,0,1,0,0,0,0,0,0
4,0.140178,-0.149865,0,0,0,0,0,0,0,1,0,0,0


##Training##

Split Train Set in Train  & Validation Sets(70/30)

In [57]:
############# inputs to the neural network #######################
train_set = z.sample(frac=0.7,random_state=200)
validation_set = z.drop(train_set.index)
######################### normalized target outputs for training set##########
y_train_set = train_set.pop('Price')
y_train_set_mean = y_train_set.mean()
y_train_set_std = y_train_set.std()
y_train_set = (y_train_set - y_train_set_mean) / y_train_set_std
########################## normalized target outputs for validation set #######
y_validation_set = validation_set.pop('Price')
y_validation_set_mean = y_validation_set.mean()
y_validation_set_std = y_validation_set.std()
y_validation_set = (y_validation_set - y_validation_set_mean) / y_validation_set_std

Import tensorflow

In [60]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


Build model/neural network

In [93]:
model = keras.Sequential([
    keras.layers.Dense(52,input_shape=(13,)),  # input layer (1)
    keras.layers.Dense(52, activation='sigmoid'),  # hidden layer (2)
    keras.layers.Dense(1, activation='sigmoid') # output layer (3)
])

Compile model

In [94]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['accuracy'])

Model Training

In [92]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_17 (Dense)             (None, 13)                182       
_________________________________________________________________
dense_18 (Dense)             (None, 52)                728       
_________________________________________________________________
dense_19 (Dense)             (None, 1)                 53        
Total params: 963
Trainable params: 963
Non-trainable params: 0
_________________________________________________________________


In [95]:
model.fit(train_set, y_train_set, epochs=100) 

Epoch 1/100
137/137 [==============================] - 1s 1ms/step - loss: 1.0140 - accuracy: 0.0000e+00
Epoch 2/100
137/137 [==============================] - 0s 1ms/step - loss: 0.8759 - accuracy: 0.0000e+00
Epoch 3/100
137/137 [==============================] - 0s 1ms/step - loss: 0.8534 - accuracy: 0.0000e+00
Epoch 4/100
137/137 [==============================] - 0s 1ms/step - loss: 0.9498 - accuracy: 0.0000e+00
Epoch 5/100
137/137 [==============================] - 0s 1ms/step - loss: 0.8800 - accuracy: 0.0000e+00
Epoch 6/100
137/137 [==============================] - 0s 1ms/step - loss: 0.9641 - accuracy: 0.0000e+00
Epoch 7/100
137/137 [==============================] - 0s 1ms/step - loss: 0.8883 - accuracy: 0.0000e+00
Epoch 8/100
137/137 [==============================] - 0s 1ms/step - loss: 0.7966 - accuracy: 0.0000e+00
Epoch 9/100
137/137 [==============================] - 0s 1ms/step - loss: 0.9073 - accuracy: 0.0000e+00
Epoch 10/100
137/137 [==============================] -